# مثال على استخدام أداة التوليد التلقائي


## استيراد الحزم اللازمة


In [ ]:
import os
import json

import requests
from autogen_agentchat.agents import AssistantAgent
from autogen_core.models import UserMessage
from autogen_ext.models.azure import AzureAIChatCompletionClient
from azure.core.credentials import AzureKeyCredential
from autogen_core import CancellationToken
from autogen_core.tools import FunctionTool
from autogen_agentchat.messages import TextMessage
from autogen_agentchat.ui import Console
from typing import Any, Callable, Set, Dict, List, Optional

## إنشاء العميل

في هذا المثال، سنستخدم [GitHub Models](https://aka.ms/ai-agents-beginners/github-models) للوصول إلى نموذج اللغة الكبير (LLM).

يتم تعريف `model` كـ `gpt-4o-mini`. جرّب تغيير النموذج إلى نموذج آخر متاح في سوق GitHub Models لمشاهدة النتائج المختلفة.

كمحاولة سريعة، سنقوم فقط بتشغيل موجه بسيط - `ما هي عاصمة فرنسا`.


In [ ]:
client = AzureAIChatCompletionClient(
    model="gpt-4o-mini",
    endpoint="https://models.inference.ai.azure.com",
    # To authenticate with the model you will need to generate a personal access token (PAT) in your GitHub settings.
    # Create your PAT token by following instructions here: https://docs.github.com/en/authentication/keeping-your-account-and-data-secure/managing-your-personal-access-tokens
    credential=AzureKeyCredential(os.environ["GITHUB_TOKEN"]),
    model_info={
        "json_output": True,
        "function_calling": True,
        "vision": True,
        "family": "unknown",
    },
)

result = await client.create([UserMessage(content="What is the capital of France?", source="user")])
print(result)

## تعريف الوظائف

في هذا المثال، سنمنح الوكيل إمكانية الوصول إلى أداة وهي عبارة عن وظيفة تحتوي على قائمة بوجهات العطلات المتاحة وتوافرها.

يمكنك أن تتخيل أن هذا سيكون سيناريو حيث قد يكون لدى وكيل السفر إمكانية الوصول إلى قاعدة بيانات للسفر على سبيل المثال.

أثناء استعراضك لهذا المثال، لا تتردد في محاولة تعريف وظائف وأدوات جديدة يمكن للوكيل استدعاؤها.


In [ ]:
from typing import Dict, List, Optional


def vacation_destinations(city: str) -> tuple[str, str]:
    """
    Checks if a specific vacation destination is available
    
    Args:
        city (str): Name of the city to check
        
    Returns:
        tuple: Contains city name and availability status ('Available' or 'Unavailable')
    """
    destinations = {
        "Barcelona": "Available",
        "Tokyo": "Unavailable",
        "Cape Town": "Available",
        "Vancouver": "Available",
        "Dubai": "Unavailable",
    }

    if city in destinations:
        return city, destinations[city]
    else:
        return city, "City not found"

# Example usage:
# city, status = vacation_destinations("Barcelona")
# print(f"How about visiting {city}? It's currently {status} there!")

## تعريف أداة الوظيفة
لكي يتمكن الوكيل من استخدام `vacation_destinations` كـ `FunctionTool`، نحتاج إلى تعريفها كواحدة.

سنقوم أيضًا بتوفير وصف للأداة، مما يساعد الوكيل على التعرف على الغرض من استخدام هذه الأداة فيما يتعلق بالمهمة التي طلبها المستخدم.


In [ ]:
get_vacations = FunctionTool(
    vacation_destinations, description="Search for vacation destinations and if they are available or not."
)

## تعريف الوكيل

الآن يمكننا إنشاء الوكيل في الكود أدناه. نقوم بتعريف `system_message` لإعطاء الوكيل تعليمات حول كيفية مساعدة المستخدمين في العثور على وجهات العطلات.

كما نقوم بتعيين المعامل `reflect_on_tool_use` إلى true. يتيح هذا استخدام نموذج اللغة الكبير (LLM) لأخذ استجابة استدعاء الأداة وإرسال الرد باستخدام اللغة الطبيعية.

يمكنك تعيين المعامل إلى false لملاحظة الفرق.


In [ ]:
agent = AssistantAgent(
    name="assistant",
    model_client=client,
    tools=[get_vacations],
    system_message="You are a travel agent that helps users find vacation destinations.",
    reflect_on_tool_use=True,
)

## تشغيل الوكيل

الآن يمكننا تشغيل الوكيل مع الرسالة الأولية من المستخدم التي تطلب القيام برحلة إلى طوكيو.

يمكنك تغيير وجهة المدينة لمعرفة كيف يستجيب الوكيل لتوافر المدينة.


In [ ]:
async def assistant_run() -> None:
    response = await agent.on_messages(
        [TextMessage(content="I would like to take a trip to Tokyo", source="user")],
        cancellation_token=CancellationToken(),
    )
    print(response.inner_messages)
    print(response.chat_message)


# Use asyncio.run(assistant_run()) when running in a script.
await assistant_run()


---

**إخلاء المسؤولية**:  
تمت ترجمة هذا المستند باستخدام خدمة الترجمة الآلية [Co-op Translator](https://github.com/Azure/co-op-translator). بينما نسعى لتحقيق الدقة، يرجى العلم أن الترجمات الآلية قد تحتوي على أخطاء أو عدم دقة. يجب اعتبار المستند الأصلي بلغته الأصلية هو المصدر الموثوق. للحصول على معلومات حساسة أو هامة، يُوصى بالاستعانة بترجمة بشرية احترافية. نحن غير مسؤولين عن أي سوء فهم أو تفسيرات خاطئة تنشأ عن استخدام هذه الترجمة.
